Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [38]:

library('tidymodels')
library('ISLR2')
library('tidyverse')



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ lubridate 1.9.3     ✔ stringr   1.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor() masks scales::col_factor()
✖ purrr::discard()    masks scales::discard()
✖ dplyr::filter()     masks stats::filter()
✖ stringr::fixed()    masks recipes::fixed()
✖ dplyr::lag()        masks stats::lag()
✖ readr::spec()       masks yardstick::spec()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [40]:
Default |> head()

default student balance   income   
1 No      No       729.5265 44361.625
2 No      Yes      817.1804 12106.135
3 No      No      1073.5492 31767.139
4 No      No       529.2506 35704.494
5 No      No       785.6559 38463.496
6 No      Yes      919.5885  7491.559

We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [41]:
Default |> count(default)

default n   
1 No      9667
2 Yes      333

The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [42]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [43]:

mod = logistic_reg() |> set_engine('glm') |> set_mode('classification')



Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [46]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric  .estimator .estimate
1 accuracy binary     0.9666667

Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [47]:

folds = vfold_cv(Default, v = 10, strata = 'default')

folds |> glimpse()



Rows: 10
Columns: 2
$ splits <list> [<vfold_split[9000 x 1000 x 10000 x 4]>], [<vfold_split[9000 x…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [48]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric     .estimator mean       n  std_err     .config             
1 accuracy    binary     0.97350000 10 0.001627882 Preprocessor1_Model1
2 brier_class binary     0.02147823 10 0.001099511 Preprocessor1_Model1
3 roc_auc     binary     0.94839630 10 0.004832559 Preprocessor1_Model1

❓How does the model accuracy compare to the naive model from above?


The model accuracy (0.97350000) is slightly more accurate than the accuracy of the naive model.  


Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [56]:
rec = recipe(default ~ income + balance + student, data=Default) |> step_dummy(student)



mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

.metric     .estimator mean       n  std_err     .config             
1 accuracy    binary     0.97280000 10 0.001685230 Preprocessor1_Model1
2 brier_class binary     0.02140828 10 0.001147187 Preprocessor1_Model1
3 roc_auc     binary     0.94893241 10 0.005105788 Preprocessor1_Model1

❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?


Including student as a predictor does causes the model accuracy to decreases. 


Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [57]:

rec = recipe(default ~ income + balance, data=Default_test)



mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()


.metric     .estimator mean       n  std_err     .config             
1 accuracy    binary     0.97350000 10 0.001627882 Preprocessor1_Model1
2 brier_class binary     0.02147823 10 0.001099511 Preprocessor1_Model1
3 roc_auc     binary     0.94839630 10 0.004832559 Preprocessor1_Model1


Yes, but only slightly.
